## 024-Genome Assembly as Shortest Superstring

### ゲノムシーケンス入門

同一ゲノムの複数コピーを小さな断片（リード）に分割し、それらを計算機的に再構築することでゲノム配列を復元する。

「GC含量の計算」で見たように、人間のゲノム配列の約99.9%はほぼ共通している。したがって、もしある種の完全長ゲノムがいくつか解明されれば、その種のゲノム全体を解読する重要な鍵をすでに手にしていることになる。

生物の完全なゲノム配列（ゲノムシーケンス）を決定することは、バイオインフォマティクスの中心的課題である。しかし、我々は依然として「顕微鏡で一塩基ごとにズームして読み取る」ような技術は持っていない。その代わりに、研究者は化学的方法を用いて、より小さなDNA断片（リード）を生成・同定することができる。

大量のリードを同一ゲノムの複数コピーから得たのち、目的とするゲノムをそれらの断片から再構築する。この過程が**フラグメントアセンブリ**である（図1参照）。

---

### 問題設定

一群の文字列が与えられたとき、それらすべてを部分文字列として含むより大きな文字列を**スーパー文字列**という。

最小仮定の原則（パーシモニー）に基づけば、リード集合に対する最短スーパー文字列が候補染色体となる。

**与えられるもの**  
- 高々50本のDNA文字列（長さはほぼ等しく、最大1 kbpまで）。  
- FASTA形式。  
- すべて同一の線状染色体の同一鎖由来のリードと仮定。  

**保証される条件**  
- リードを「長さの半分を超えて重なる部分」でつなぎ合わせることで、全染色体を一意に復元できる。

**求めるもの**  
- 与えられたすべての文字列を含む最短スーパー文字列（すなわち復元染色体配列）。

---

### サンプルデータセット

```
>Rosalind_56
ATTAGACCTG
>Rosalind_57
CCTGCCGGAA
>Rosalind_58
AGACCTGCCG
>Rosalind_59
GCCGGAATAC
```

### サンプル出力

```
ATTAGACCTGCCGGAATAC
```

---

### 補足情報

フラグメントアセンブリの目標は完全なゲノムを得ることであるが、実際には染色体のいくつかの連続領域（コンティグ）を構築するにとどまる。また、すべてのリードが同一の鎖由来であるという仮定は現実的ではなく、実際の研究では各リードがどのDNA鎖から得られたかは不明である。


In [ ]:
def parse_fasta(lines):
    fasta_dict = {}
    current_key = None
    current_seq = []

    for line in lines:
        if line.startswith(">"):
            if current_key:
                fasta_dict[current_key] = "".join(current_seq)
            current_key = line[1:]
            current_seq = []
        else:
            current_seq.append(line)

    # 最後のレコードの追加
    if current_key:
        fasta_dict[current_key] = "".join(current_seq)

    return fasta_dict

In [77]:
fasta = """>Rosalind_56
ATTAGACCTG
>Rosalind_57
CCTGCCGGAA
>Rosalind_58
AGACCTGCCG
>Rosalind_59
GCCGGAATAC
"""

In [ ]:
fasta_dict = parse_fasta(fasta.splitlines())
reads = list(fasta_dict.values())

In [ ]:
print(len(reads))
print(reads)

1
['ATTAGACCTGCCGGAATAC']


In [ ]:
def _overlap_len_suffix_prefix(a: str, b: str) -> int:
    """aの末尾とbの先頭の最大一致長を返す（例：a[-k:] == b[:k] を満たす最大k）。"""
    max_k = min(len(a), len(b))
    for k in range(max_k, 0, -1):  # 長い方から先に確認し、見つかり次第返す
        if a[-k:] == b[:k]:
            return k
    return 0


def _merge_with_max_overlap(a: str, b: str) -> tuple[int, str]:
    """a→b, b→a の両方向で重なり長を比較し、重なりが大きい方で結合して返す。"""
    k_ab = _overlap_len_suffix_prefix(a, b)
    k_ba = _overlap_len_suffix_prefix(b, a)

    if k_ab >= k_ba:
        # aの末尾とbの先頭がk_abだけ一致 → a + bの非重複部分
        return k_ab, a + b[k_ab:]
    else:
        # bの末尾とaの先頭がk_baだけ一致 → b + aの非重複部分
        return k_ba, b + a[k_ba:]


def _find_best_pair(reads: list[str]) -> tuple[int, int, str]:
    """
    現在のreadsから「最も重なるペア」を1組見つける。
    戻り値: (i, j, overlap_len, merged_string)
    """
    best_i = best_j = -1
    best_overlap = -1
    best_merged = ""

    n = len(reads)
    for i in range(n):
        for j in range(i + 1, n):
            overlap_len, merged = _merge_with_max_overlap(reads[i], reads[j])
            if overlap_len > best_overlap:
                best_overlap = overlap_len
                best_merged = merged
                best_i, best_j = i, j

    return best_i, best_j, best_merged


def shortest_superstring(reads: list[str]) -> str:
    """
    最短スーパー文字列を貪欲法で作る。
    手順：
      1) 全ペアから最大オーバーラップを持つ2本を選ぶ
      2) その2本を結合して1本にする
      3) 1本になるまで繰り返す
    備考：
      - Rosalind LONGの前提（>半長オーバーラップで一意復元）があるため、この貪欲法で正解に到達できる。
    """

    while len(reads) > 1:
        i, j, best_merged = _find_best_pair(reads)

        # 見つけた最良ペアを結合で置き換え、片方を削除
        reads[i] = best_merged
        reads.pop(j)

    return reads[0]

In [ ]:
assert shortest_superstring(reads) == "ATTAGACCTGCCGGAATAC"

## データセット

In [91]:
from pathlib import Path
fasta = Path("rosalind_long.txt").read_text()

In [92]:
fasta_dict = parse_fasta(fasta.splitlines())
reads = list(fasta_dict.values())

In [93]:
print(len(reads))
print(reads[:2])

50
['GTGTCATGTAAGCTCGCTAGAGCTCCACCGACTCCCCCAGGCGCACGTAGGGTTATTCACGGATATGCAATGGCTCGATAGGGCGTTATAGACCGCACCCAATTTATCTCCATCCACTAGCGTCGTGCCGTAACTCATAGATTGGTTTCGAGTCTTACGGTGAACACTCGCCGCTCACGAATCCGCAGCACAGGATCGAGCAACAGTAAGGTCCCGGTAACAGTTGCCCTTTTCACCTATCTCCACTCTCCGACCACAACTCCCGTAAGCATCTAAAGCCTAGAACTCTTGGGGAGAGACGAAGCACTTAATCCGTCCACGGATATTGGTCCAACGTGTGCATATGGGCCCCAGGGAGTCAGACGAGAATCGCCGTTTCTAGAGTAGATGGGCTTAAGTCAGCGCGCTCTTCACCGGGTAAGGACGCGTTGTATGCCCATCACTCTTTTTATCCGTGTGGTTTTAGATGGACCTGCGTGGTTGGGCGCAGGCGGGGGTGTCAGGGATGTCTGATTCAAGCGGTATGTTCGGAGAACTGCAGGCGCGGTCTGCCATAGTTGTGTACATTTCTGGTGGACACATTGCGAATTATGGACATGCTGCGTTCTTCGGGGGCGGTGCCTGACTAAGGAACCTGCACCCCGGGGTGCATCTCGTTTCGGCTCGCGCCAGATTTGTCGTGTGACCACACGCGGCTGTCATCAACTAGACCCGGATAGTTACTATTAAGCGAATGGCGAGATCGACGGCTACATGGAAGAACGAATTGCGAAGGAAGCCCAGGGCGAATCCTCGTACACGTGCCGCACGCTGCCTCCGTCTTCTCTTGGATGCCCAAAGCTCCTCAAGTGGGGTTCAGAACCTGTTATAAACCCCTGCGGGGTCATTTAGGGCGCTTTTGGCACTTTACGTTGTAAATGTATCTAACCAAACCGACGCTCCATCCGCGATATATATACGCTCCCTTTCTTTATCTCCGGACGAATCGTGTGACC

In [94]:
ans = shortest_superstring(reads)
print(ans)

GCGGCCGGTAAAAGTTAGGGATGTCAAGAGGATGCCGCCTAGGAATCAGACTGTTTGGCGCCGTATACAGTCTAGAACTAGCGCCCCCCGCACCCATAGTTCCAAGCTACCTTACTGCTGAACAAGCAATTGGGTCGCGTATACGATATGAATTGTCCGTTACTTCCAATGAGAGCCCGTCCGTCGTCAAGGGGGGAACTTACGTTGGTTCCTCTTAAGCCTCTTATACCCCGCCAAAGCGGGATGGCGTACAATTTCTGCAGGATTGCATATTCTTACTTTTATCATTTCCTATGTAGACCCATGGAGATGTGGTACTGAATAGGTCTCTGGTCTGGCAGTAGTCCACGCATTTCCCAGGGGAAGGGGACAGATTATACCCGCCTAGAGTCAAAATACCGCCTCGATATTAAATCCGATTGATGCTTGAGGTGCCGGCGGTCCTCGCGTGCATAATGACTGTGCTGTCTGCCTGGTACGTAGCTTGTGAAAGCCCTGTATTGGGCCATAGATGGCAAATGGCACTACCACTTTCACTACCACTAAGCCAACTGTGCAAGTACCGAAGGAACCACCGATGCGAGTAAGACGTATAAAACTGTCAAGATTTCCCAGATCTAGTCTGAGAGTTTTTGGCAGGCTTTGACTCTTTCGTATGCACGGATGCAACCTTATTCTCCCCAAACACAAAATCGCCTCGCCCGACTGTTTGCACGTATTGCAAGATTCGACCCTTACTGCCGTCGGGAACATTCCGCTTGCCGTGGCGAGACAAGCTCTTATAGAAGTGCCACATGTCAGAGGCAAAGCGGAGCTATAAGCGAACAATCACAGTCAACCTTCACTCATAGCGGTCCTCGGGTTACTGACCTTTTTTAATGATAATTGCGCCTGATTATCTTCGAATACTTAGAGAGTGTTATGCCGCCGTCTGATCATGCTTCGGAGGGTATGTGGGTGGCGCGTTCTTGAGCCCGTACGTGTTCTTGGTCAAACCCTC